<a href="https://colab.research.google.com/github/caprolaliac/image_processing_YOLOv8/blob/main/yolotest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'detect-footpath-vendors:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4636682%2F7896228%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240402%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240402T051737Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3De08c4bf76b56f8be60778d2c1f04c75fff8454a224ac1baa2b5ab00ae17ac0c5781967de09f6e4878b640abb0a3ed98af0b04ac1ad0373c5db1895f4cbc1271f027374453ef8e8439b8dc5f951860abba6d199bc6b2a7beebf08b0c58f28c70dc1fdac6a13a1bc045599823a9558a736ff55e6a3332f14aa0e86f489ef92498cc63cba82c9dd4d1b6be434604ff2d2e392dc0ea4bcafbe61772cd6492dc8d21c8b08f8e6cb3da05bd31cb36aa1337c57dacbd15947de5ef0360221519c2acd9a7ffba2dfaa35eb6c4b3539d115e518992c1294f8df3acb0abb7ecd51d78d99c48c2e62e7592373b92dfa7a3df38ca759ff5d53e2df0734814dcc952d50801be4'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
!pip install ultralytics -U

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8x-obb.pt')

results = model('/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/58_jpg.rf.5ff4a76b1f42864e3eac3b9102fdaef2.jpg',imgsz=640, save=True)  # predict on an image


image 1/1 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/58_jpg.rf.5ff4a76b1f42864e3eac3b9102fdaef2.jpg: 640x640 (no detections), 77.6ms
Speed: 1.7ms preprocess, 77.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/obb/predict3


In [ ]:
model = YOLO('yolov8x-obb.pt')

results = model.train(data='/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/data.yaml', epochs=100, imgsz=640)

Ultralytics YOLOv8.1.40 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8x-obb.pt, data=/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False,

wandb: Currently logged in as: gvgvgv033 (veghotdog). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/train/labels... 180 images, 1 backgrounds, 0 corrupt: 100%|██████████| 180/180 [00:00<00:00, 603.54it/s]


train: WARNING ⚠️ Cache directory /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all         52        240      0.526      0.236      0.218      0.143



100 epochs completed in 0.518 hours.
Optimizer stripped from runs/obb/train2/weights/last.pt, 139.6MB
Optimizer stripped from runs/obb/train2/weights/best.pt, 139.5MB

Validating runs/obb/train2/weights/best.pt...
Ultralytics YOLOv8.1.40 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69488574 parameters, 0 gradients, 263.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


                   all         52        240      0.652      0.232      0.253      0.159
                  Auto         52          5      0.626        0.4       0.51      0.414
             Barricade         52          1          1          0      0.166     0.0995
                  Bike         52         21      0.485      0.449       0.53      0.337
                   Bus         52          5      0.359      0.231      0.296      0.178
                   Car         52         19      0.457      0.399      0.425      0.307
  Distress in footpath         52          2          0          0    0.00949    0.00285
                   Dog         52          2          1          0          0          0
         Electric Pole         52          3          1          0      0.227      0.127
         Electric pole         52          7      0.232      0.143       0.18     0.0939
           Pedestrians         52         14      0.373        0.5      0.279      0.151
          Road divide

lr/pg0,▂▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg1,▂▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
lr/pg2,▂▃▆████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▂▃▄▄▄▆▅▇▇▇▆▇▆▆▇▇▇█▇▇▆▇▇▇█▇▇▆▇▇▆▇▇▇▇▇▇▇█
metrics/mAP50-95(B),▁▂▂▃▃▄▆▅▆▆▆▆▇▆▆▇▇██▇▇▆▇▇▇█▇▇▇▇█▇▇▇▇▇▇▇▇█
metrics/precision(B),▁▁▆█▇▆█▇▇▅▆▆▆▆▇▄▅▆▇▇▆▇▆▇▆▇▆█▇█▇▇▆▆▆▆▆▆▆▇
metrics/recall(B),▁▆▄▃▃▄▄▅▆▆█▅▇▆▆▇▇▇▇▇▇▆▇▆█▇█▇▆▆▇▆▇▇▇▇▇▇▇▇
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▄▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁


In [ ]:
from ultralytics import YOLO

model = YOLO('runs/obb/train/weights/best.pt')

metrics = model.val(data='/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/data.yaml')

print(metrics.box.map)
print(metrics.box.map50)
print(metrics.box.map75)
print(metrics.box.maps)

Ultralytics YOLOv8.1.40 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69488574 parameters, 0 gradients, 263.4 GFLOPs


val: Scanning /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/valid/labels... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<00:00, 678.97it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.24s/it]


                   all         52        240      0.652      0.232      0.253      0.159
                  Auto         52          5      0.624        0.4       0.51      0.414
             Barricade         52          1          1          0      0.166     0.0995
                  Bike         52         21      0.487      0.453      0.534      0.338
                   Bus         52          5       0.36      0.232      0.296      0.178
                   Car         52         19      0.456      0.398      0.425      0.309
  Distress in footpath         52          2          0          0    0.00949    0.00285
                   Dog         52          2          1          0          0          0
         Electric Pole         52          3          1          0      0.227      0.127
         Electric pole         52          7      0.232      0.143       0.18     0.0938
           Pedestrians         52         14      0.373        0.5      0.278      0.151
          Road divide

In [ ]:
model = YOLO('runs/obb/train/weights/best.pt')

results = model('/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images',imgsz=640, save=True)


image 1/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/101_jpg.rf.6453328315bfee1c4e95e7753d554e86.jpg: 640x640 77.3ms
image 2/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/113_jpg.rf.0ecfe64de3295e85f0d5ee8569f8b8ca.jpg: 640x640 63.7ms
image 3/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/116_jpg.rf.930f200db12295dbd244162295ad53a1.jpg: 640x640 63.6ms
image 4/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/11_jpg.rf.8d130c870b616dd2126458a1c85b3f6d.jpg: 640x640 56.5ms
image 5/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/14_jpg.rf.7a78a90c0362070617b09fbf5ddbbad7.jpg: 640x640 57.7ms
image 6/40 /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images/16_jpg.rf.f81456c6ee659c8a5e01fa2878c90d1a.jpg: 640x640 58.1ms
image 7/40 /kaggle

In [ ]:
!ls runs/obb/train/weights

best.pt  last.pt


In [ ]:
import shutil

# Source path of the best.pt file
source_path = '/kaggle/working/runs/obb/train/weights/best.pt'

# Destination path to copy the file within Kaggle environment
destination_path = '/kaggle/working/best.pt'

# Copy the file to the destination path
shutil.copyfile(source_path, destination_path)

# Download the best.pt file
from IPython.display import FileLink
FileLink('/kaggle/working/best.pt')


/kaggle/working/best.pt

In [ ]:
model = YOLO('runs/obb/train/weights/best.pt')
metrics = model.val(data='/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/data.yaml')

mAP = metrics.box.map

mAP_50 = metrics.box.map50


mAP_75 = metrics.box.map75


mAPs = metrics.box.maps


precision = metrics.box.p
recall = metrics.box.r

overall_precision = metrics.box.mp
overall_recall = metrics.box.mr

print(f"mAP: {mAP:.4f}")
print(f"mAP@0.5: {mAP_50:.4f}")
print(f"mAP@0.75: {mAP_75:.4f}")
print(f"mAPs: {mAPs}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Overall Precision: {overall_precision:.4f}")
print(f"Overall Recall: {overall_recall:.4f}")

Ultralytics YOLOv8.1.40 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-obb summary (fused): 287 layers, 69488574 parameters, 0 gradients, 263.4 GFLOPs


val: Scanning /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/valid/labels... 52 images, 0 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<00:00, 649.56it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/valid is not writeable, cache not saved.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.25s/it]


                   all         52        240      0.652      0.232      0.253      0.159
                  Auto         52          5      0.624        0.4       0.51      0.414
             Barricade         52          1          1          0      0.166     0.0995
                  Bike         52         21      0.487      0.453      0.534      0.338
                   Bus         52          5       0.36      0.232      0.296      0.178
                   Car         52         19      0.456      0.398      0.425      0.309
  Distress in footpath         52          2          0          0    0.00949    0.00285
                   Dog         52          2          1          0          0          0
         Electric Pole         52          3          1          0      0.227      0.127
         Electric pole         52          7      0.232      0.143       0.18     0.0938
           Pedestrians         52         14      0.373        0.5      0.278      0.151
          Road divide

In [ ]:
model = YOLO('runs/obb/train/weights/best.pt')

results = model('/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/test/images',imgsz=640, save=True)

**2**

In [ ]:
!pip show pyyaml tensorflow

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!nvcc --version
!python --version

In [ ]:
!pip install torch torchvision -f https://download.pytorch.org/whl/cu101/torch_stable.html

In [ ]:
!pip install git+https://github.com/facebookresearch/fvcore

!pip install cython; pip install git+https://github.com/philferriere/cocoapi.git


In [ ]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git

In [ ]:
!pip install detectron2


In [ ]:
!pip install torch torchvision torchaudio cython matplotlib
!git clone https://github.com/facebookresearch/detectron2.git


In [ ]:
!pip install antlr4-python3-runtime==4.9.3 black==24.3.0 detectron2==0.6 fvcore==0.1.5.post20221221 hydra-core==1.3.2 iopath==0.1.9 omegaconf==2.3.0 packaging==24.0 pathspec==0.12.1 pycocotools==2.0.7

In [ ]:
!python -m pip install -e detectron2

In [ ]:
# Reload sys module to pick up changes to environment variables
import sys

# Import installed detectron2 (adjust path if needed)
sys.path.append('/kaggle/working/detectron2')  # Replace with your installation path
import detectron2



In [ ]:

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

In [ ]:
cfg = get_cfg()
cfg.merge_from_file("/kaggle/input/detect-footpath-vendors/Detect footpath vendors.v6i.yolov8-obb/data.yaml")